# Bell State Stabilization of Superconducting Circuits with Real-time Feedback
This script, based on [zhinst-qcodes](https://github.com/zhinst/zhinst-qcodes), demonstrates how to realize **Bell state stabilization** using the Lookup Table Decoder of the PQSC. For this purpose, we detect arbitrary errors on a pair of entangled data qubits in a Bell state by reading out another pair of ancilla qubits using UHFQA. The readout results are forwarded to the PQSC for further processing and generation of feedback decision. The decision is then communicated to the HDAWGs and used for waveform selection to send out the necessary control signals to restore both the data qubits and ancilla qubits back to their original states. The concept is discussed in detail in the relative [blog post](https://blogs.zhinst.com/bahadir/2021/08/18/bell-state-stabilization-of-superconducting-qubits-with-real-time-feedback/).

Let’s dive in now to see how it works! 



Copyright (C) 2021-2022 Zurich Instruments
This software may be modified and distributed under the terms of the MIT license. See the LICENSE file for details.

## Table of Contents:
* [1. Experimental Setup](#setup)
* [2. Import Packages](#import)
* [3. Initialization](#initialization)
* [4. Readout Parameters](#readout)
* [5. PQSC Configuration](#pqsc_config)
* [6. HDAWG Configuration](#hdawg_config)
    * [6.1 Configuration of HDAWG 1 AWG 0 for Control of Data Qubit D<sub>1</sub>](#hdawg1_awg0_config)
    * [6.2 Configuration of HDAWG 2 AWG 0 for Control of Data Qubit D<sub>2</sub>](#hdawg2_awg0_config)
    * [6.3 Configuration of HDAWG 1 AWG 1 for Control of Z-Ancilla](#hdawg1_awg1_config)
    * [6.4 Configuration of HDAWG 2 AWG 1 for Control of X-Ancilla](#hdawg2_awg1_config)
* [7. UHFQA Configuration](#uhfqa_config)
* [8. Main Program](#main)

## 1. Experimental Setup: <a class="anchor" id="setup"></a>
Here is the required experimental setup, including all the wiring.

![The setup](experimental_setup.svg)

The assignments of the devices for certain tasks are as follows:

![Bell state stabilization](channel_assignments.svg)

### 2. Import Packages <a class="anchor" id="import"></a>

Import zhinst-qcodes drivers

In [ ]:
zhinst_qcodes_min_version = "0.3.2"
from packaging import version
from zhinst.qcodes import __version__ as zhinst_qcodes_version
if version.parse(zhinst_qcodes_version) < version.parse(zhinst_qcodes_min_version):
    raise ImportError(f'zhinst-qcodes must be at least version {zhinst_qcodes_min_version:s}')

In [ ]:
from zhinst.qcodes import HDAWG, UHFQA, PQSC, CommandTable, Waveforms

Import other external packages

In [ ]:
import numpy as np
import textwrap
import itertools

### 3. Initialization <a class="anchor" id="initialization"></a>

* Set the data server host address
* The serial numbers of the instruments to run the script with

In [ ]:
# Dataserver host
dataserver_host = "localhost"

# Instrument serials
serial_hdawg_1 = "dev8001"
serial_hdawg_2 = "dev8002"
serial_uhfqa = "dev2001"
serial_pqsc = "dev10001"

* Connect the instruments to data server and initialize their drivers

In [ ]:
# Connect the instruments to data server
hdawg_1 = HDAWG(serial_hdawg_1, name="hdawg1", host=dataserver_host)
hdawg_2 = HDAWG(serial_hdawg_2, name="hdawg2", host=dataserver_host)
uhfqa = UHFQA(serial_uhfqa, name="uhfqa", host=dataserver_host)
pqsc = PQSC(serial_pqsc, name="pqsc", host=dataserver_host)

#get the session. it's the same for all instruments
session = pqsc.session

* To create a base configuration, load factory preset to all devices

In [ ]:
# Load factory default settings to all devices
hdawg_1.factory_reset()
hdawg_2.factory_reset()
uhfqa.factory_reset()
pqsc.factory_reset()

### 4. Readout Parameters <a class="anchor" id="readout"></a>

Necessary parameters to be defined for multiplexed readout of 2 ancilla qubits and storing the results in the register bank of PQSC are:
* Readout frequencies 
* Additional phase offset added to I and Q signals for simulated qubit responses
* Pulse length of the readout tones
* Number of samples for the raising/falling part of the gaussian flattop pulses
* Sample rate of UHFQA
* Trigger period sent out by the PQSC and feedback processing time. Note that this should be long enough to allow feedback processing to be completed.
* Register address in the PQSC register bank to store readout results

In [ ]:
# Readout frequencies
frequencies = np.array([10, 31.5]) * 1e6
# Additional phase offset added to I and Q signals for simulated qubit responses
phases = [0, 180]
# Pulse lenght of the readout tones
readout_pulse_length = 1024
# Number of samples for the raising/falling part of the gaussian flat top pulses
rise_fall_length = 100
# Sample rate of UHFQA
SR_UHFQA = 1.8e9
# Trigger period sent out from PQSC and feedback processing time (2us should be enough)
trigger_period = 2e-6
# Select register address in the PQSC to store readout results
register_num = 1

Other dependent variables:
* Waiting time for the UHFQA to allow feedback data processing after receving the start trigger from the PQSC
* Number of readout channels to use for readout of ancilla qubits (equal to the number of ancilla qubits coupled to the same feedline).
* Number of readouts (depends on the number of ancilla qubits and number of states we are simulating)

In [ ]:
# Waiting time for the UHFQA after receving the start trigger
# Divide to 16, round down, and multiply again with 16 to obtain correct waveform granularity for UHFQA
wait_samples_uhfqa = int((trigger_period * SR_UHFQA - readout_pulse_length) // 16) * 16
# Number of readout channels
num_readout_channels = len(frequencies)
# Number of times the readout is performed
num_readouts = len(phases) ** num_readout_channels

### 5. PQSC Configuration <a class="anchor" id="pqsc_config"></a>

* Check that the PQSC is warmed up and clocks are stable
* Configure the PQSC to use external reference clock
* Wait until the reference clock is locked

In [ ]:
# Verify that the PQSC has warmed up and clocks are stable
if not pqsc.system.clocks.ready():
    raise Exception("PQSC needs to warm up or clock are unstable. Try again later")

In [ ]:
# Use external reference clock.
pqsc.system.clocks.referenceclock.in_.source("external")
# Wait until locking is succesful (30 seconds timeout)
pqsc.check_ref_clock(timeout=30)

* Port numbers where HDAWGs are connected to: [HDAWG1, HDAWG2]

In [ ]:
# ZSync output ports to the receiver HDAWGs (zero based)
zsync_ports = [0, 10]

Settings related to execution engine:
* Number of triggers to be sent out. It must be equal to the number of readouts
* Trigger period sent out from PQSC and feedback process time. It is already set above together with other readout parameters. It must be long enough to allow feedback data processing.

In [ ]:
# How many triggers to send to start synchronized actions on the HDAWG and the UHFQA
pqsc.execution.repetitions(num_readouts)
# How long to wait for feedback to arrive from UHFQA
pqsc.execution.holdoff(trigger_period)

**Settings related to condition unit (LUT decoder)**

![PQSC_lut_decoder_diagram.svg](PQSC_lut_decoder_diagram.svg)

Configuration of Source Register Selector:
* The source register selector is used to reduce the large set of readout registers to an address word of 16 bits. This address word serves as the index for look-up table to access its entries.
* The user can select 16 sources by indicating register numbers and indices of the desired bit inside of each register. 
* **Important note**:  If an address bit is not configured, it is automatically set to register 0, index 0. Therefore, it is better not to use Register 0 Index 0 to store any qubit readout result. Otherwise, the adress values will be wrong.
* In the code below the Source Register Selector is configured such that:
    * Register 2, Bit 1 of Readout Register Bank → Bit 0 of Address Word
    * Register 2, Bit 2 of Readout Register Bank → Bit 1 of Address Word

In [ ]:
# Configure source register selector
pqsc.feedback.decoder.lut.sources[0].register(register_num)
pqsc.feedback.decoder.lut.sources[0].index(0)
pqsc.feedback.decoder.lut.sources[1].register(register_num)
pqsc.feedback.decoder.lut.sources[1].index(1)

* Generating a look up table and programing the LUT decoder:
     * The tables are programmed with an array of maximum size of 2<sup>16</sup> bytes. If the number of address bits that will be used is smaller than 16, it is possible to program the condition unit with an array of smaller size.
     * Since we have two ancilla qubits, there are a total of 2<sup>2</sup> = 4 possible combinations of the readout results. Hence, it is enough for our look-up table to have 4 entries. 

In [ ]:
# Generate a look up table and program LUT decoder
lut_length = 2 ** num_readout_channels
lut = range(lut_length)
lut = np.array(lut).astype(np.uint32)
pqsc.feedback.decoder.lut.tables[0].value(lut)
print(f"Look-up table content: {lut}")

* Configure table output selector for each port to select which of the four look up tables should be forwarded to the output
* Enable condition unit output forwarding for each port connected to the receving HDAWGs

In [ ]:
# Configure table output selector for each port
for port in zsync_ports:
    pqsc.zsyncs[port].output.decoder.source(0)

# Enable condition unit output forwarding for each port
for port in zsync_ports:
    pqsc.zsyncs[port].output.decoder.enable(1)

### 6. HDAWG Configuration <a class="anchor" id="hdawg_config"></a>

* Configure the HDAWGs to connect to the PQSC
* Assign the AWG cores as shown in [Experimental Setup](#setup)

In [ ]:
# Connect HDAWGs to the PQSC
hdawg_1.enable_qccs_mode()
hdawg_2.enable_qccs_mode()

# Assign the AWG cores to control the qubits acording to setup
awg_D1 = hdawg_1.awgs[0]
awg_D2 = hdawg_2.awgs[0]
awg_AZ = hdawg_1.awgs[1]
awg_AX = hdawg_2.awgs[1]

* Common sequence program to use with all AWG Cores of all HDAWGs. It waits for starting trigger and feedback trigger from PQSC  and plays the correct waveform depending on the feedback data

In [ ]:
# Sequence program
feedback_wfm_len = 1024
feedback_wfm_index = 0

hdawg_consts = f"""\
//Constants definition
const WFM_LEN = {feedback_wfm_len:d};
const WFM_INDEX = {feedback_wfm_index:d};
const REPETITIONS = {num_readouts:d};

""" 

hdawg_program = hdawg_consts + """\
//Create one waveform
wave w1 = ones(WFM_LEN);
assignWaveIndex(w1,WFM_INDEX);

repeat(REPETITIONS) {
    //Start trigger
    waitZSyncTrigger();
        
    //Feedback trigger
    waitZSyncTrigger();
    
    //Read feedback data and execute command table entry
    //depending on feedback data to play the correct waveform
    playWaveZSync(ZSYNC_DATA_PQSC_DECODER);

}
"""

Whenever there is a bit-flip error on one of the data qubits, we correct it by sending a R<sub>x</sub><sup>π</sup> pulse from AWG core 0 of HDAWG 1 to data qubit D<sub>1</sub>. Phase-flip errors, on the other hand, are corrected by sending a R<sub>z</sub><sup>π</sup> pulse from AWG core 0 of HDAWG 2 to data qubit D<sub>2</sub>.

#### 6.1. Configuration of HDAWG 1 AWG 0 for Control of Data Qubit D<sub>1</sub><a class="anchor" id="hdawg1_awg0_config"></a>

* Turn on the output channels
* Run the AWG in single mode

In [ ]:
# Enable outputs
hdawg_1.sigouts[0].on(True)
hdawg_1.sigouts[1].on(True)
# Disable rerun
awg_D1.single(True)

* Configure *mask*, *shift* and *offset* nodes to select portion of interest in the feedback data as described in the [PQSC User Manual](https://docs.zhinst.com/pdf/ziPQSC_UserManual.pdf).

In [ ]:
# Configure the ZSync DEMUX
awg_D1.zsync.decoder.mask(0b11)
awg_D1.zsync.decoder.shift(0)
awg_D1.zsync.decoder.offset(0)

* Compile the sequence program and upload it to the instrument

In [ ]:
awg_D1.load_sequencer_program(hdawg_program)

* Configure the command table to assign the correct waveform to each table entry.
* For demonstration purposes, we represent the feedback pulses as follows:
    * R<sub>x</sub><sup>π</sup> → Rectangular pulse with amplitude 1.0
    * No feedback action (identity I) → Rectangular pulse of amplitude 0.5

In [ ]:
# Command table
ct_schema = awg_D1.commandtable.load_validation_schema()
awg_D1_ct = CommandTable(ct_schema)

awg_D1_ct.table[0].waveform.index = 0
awg_D1_ct.table[0].amplitude0.value = 0.5
awg_D1_ct.table[0].amplitude1.value = 0.5

awg_D1_ct.table[1].waveform.index = 0
awg_D1_ct.table[1].amplitude0.value = 1.0
awg_D1_ct.table[1].amplitude1.value = 1.0

awg_D1_ct.table[2].waveform.index = 0
awg_D1_ct.table[2].amplitude0.value = 0.5
awg_D1_ct.table[2].amplitude1.value = 0.5

awg_D1_ct.table[3].waveform.index = 0
awg_D1_ct.table[3].amplitude0.value = 1.0
awg_D1_ct.table[3].amplitude1.value = 1.0

awg_D1.commandtable.upload_to_device(awg_D1_ct)

#### 6.2. Configuration of HDAWG 2 AWG 0 for Control of Data Qubit D<sub>2</sub> <a class="anchor" id="hdawg2_awg0_config"></a>

* Turn on the output channels
* Run the AWG in single mode

In [ ]:
# Enable outputs
hdawg_2.sigouts[0].on(True)
hdawg_2.sigouts[1].on(True)
# Disable rerun
awg_D2.single(True)

* Configure *mask*, *shift* and *offset* nodes to select portion of interest in the feedback data as described in the [PQSC User Manual](https://docs.zhinst.com/pdf/ziPQSC_UserManual.pdf).

In [ ]:
# Configure the ZSync DEMUX
awg_D2.zsync.decoder.mask(0b11)
awg_D2.zsync.decoder.shift(0)
awg_D2.zsync.decoder.offset(0)

* Compile the sequence program and upload it to the instrument

In [ ]:
awg_D2.load_sequencer_program(hdawg_program)

* Configure the command table to assign the correct waveform to each table entry.
* For demonstration purposes, we represent the feedback pulses as follows:
    * R<sub>z</sub><sup>π</sup> → Rectangular pulse with amplitude 1.0
    * No feedback action (identity I) → Rectangular pulse of amplitude 0.5

In [ ]:
# Command table
ct_schema = awg_D2.commandtable.load_validation_schema()
awg_D2_ct = CommandTable(ct_schema)

awg_D2_ct.table[0].waveform.index = 0
awg_D2_ct.table[0].amplitude0.value = 0.5
awg_D2_ct.table[0].amplitude1.value = 0.5

awg_D2_ct.table[1].waveform.index = 0
awg_D2_ct.table[1].amplitude0.value = 0.5
awg_D2_ct.table[1].amplitude1.value = 0.5

awg_D2_ct.table[2].waveform.index = 0
awg_D2_ct.table[2].amplitude0.value = -1.0
awg_D2_ct.table[2].amplitude1.value = -1.0

awg_D2_ct.table[3].waveform.index = 0
awg_D2_ct.table[3].amplitude0.value = -1.0
awg_D2_ct.table[3].amplitude1.value = -1.0

awg_D2.commandtable.upload_to_device(awg_D2_ct)

#### 6.3. Configuration of HDAWG 1 AWG 1 for Control of  Z-Ancilla <a class="anchor" id="hdawg1_awg1_config"></a>

* Turn on the output channels
* Run the AWG in single mode

In [ ]:
# Enable outputs
hdawg_1.sigouts[2].on(True)
hdawg_1.sigouts[3].on(True)
# Disable rerun
awg_AZ.single(True)

* Configure *mask*, *shift* and *offset* nodes to select portion of interest in the feedback data as described in the [PQSC User Manual](https://docs.zhinst.com/pdf/ziPQSC_UserManual.pdf).

In [ ]:
# Configure the ZSync DEMUX
awg_AZ.zsync.decoder.mask(0b11)
awg_AZ.zsync.decoder.shift(0)
awg_AZ.zsync.decoder.offset(0)

* Compile the sequence program and upload it to the instrument

In [ ]:
awg_AZ.load_sequencer_program(hdawg_program)

* Configure the command table to assign the correct waveform to each table entry.
* For demonstration purposes, we represent the feedback pulses as follows:
    * R<sub>x</sub><sup>π</sup> → Rectangular pulse with amplitude 1.0
    * No feedback action (identity I) → Rectangular pulse of amplitude 0.5

In [ ]:
# Command table
ct_schema = awg_AZ.commandtable.load_validation_schema()
awg_AZ_ct = CommandTable(ct_schema)

awg_AZ_ct.table[0].waveform.index = 0
awg_AZ_ct.table[0].amplitude0.value = 0.5
awg_AZ_ct.table[0].amplitude1.value = 0.5

awg_AZ_ct.table[1].waveform.index = 0
awg_AZ_ct.table[1].amplitude0.value = 1.0
awg_AZ_ct.table[1].amplitude1.value = 1.0

awg_AZ_ct.table[2].waveform.index = 0
awg_AZ_ct.table[2].amplitude0.value = 0.5
awg_AZ_ct.table[2].amplitude1.value = 0.5

awg_AZ_ct.table[3].waveform.index = 0
awg_AZ_ct.table[3].amplitude0.value = 1.0
awg_AZ_ct.table[3].amplitude1.value = 1.0

awg_AZ.commandtable.upload_to_device(awg_AZ_ct)

#### 6.4. Configuration of HDAWG 2 AWG 1 for Control of  X-Ancilla <a class="anchor" id="hdawg2_awg1_config"></a>

* Turn on the output channels
* Run the AWG in single mode

In [ ]:
# Enable outputs
hdawg_2.sigouts[2].on(True)
hdawg_2.sigouts[3].on(True)
# Disable rerun
awg_AX.single(True)

* Configure *mask*, *shift* and *offset* nodes to select portion of interest in the feedback data as described in the [PQSC User Manual](https://docs.zhinst.com/pdf/ziPQSC_UserManual.pdf).

In [ ]:
# Configure the ZSync DEMUX
awg_AX.zsync.decoder.mask(0b11)
awg_AX.zsync.decoder.shift(0)
awg_AX.zsync.decoder.offset(0)

* Compile the sequence program and upload it to the instrument

In [ ]:
awg_AX.load_sequencer_program(hdawg_program)

* Configure the command table to assign the correct waveform to each table entry.
* For demonstration purposes, we represent the feedback pulses as follows:
    * R<sub>x</sub><sup>π</sup> → Rectangular pulse with amplitude 1.0
    * No feedback action (identity I) → Rectangular pulse of amplitude 0.5

In [ ]:
# Command table
ct_schema = awg_AX.commandtable.load_validation_schema()
awg_AX_ct = CommandTable(ct_schema)

awg_AX_ct.table[0].waveform.index = 0
awg_AX_ct.table[0].amplitude0.value = 0.5
awg_AX_ct.table[0].amplitude1.value = 0.5

awg_AX_ct.table[1].waveform.index = 0
awg_AX_ct.table[1].amplitude0.value = 0.5
awg_AX_ct.table[1].amplitude1.value = 0.5

awg_AX_ct.table[2].waveform.index = 0
awg_AX_ct.table[2].amplitude0.value = 1.0
awg_AX_ct.table[2].amplitude1.value = 1.0

awg_AX_ct.table[3].waveform.index = 0
awg_AX_ct.table[3].amplitude0.value = 1.0
awg_AX_ct.table[3].amplitude1.value = 1.0

awg_AX.commandtable.upload_to_device(awg_AX_ct)

### 7. UHFQA Configuration <a class="anchor" id="uhfqa_config"></a>

* Configure the UHFQA to connect to the PQSC

In [ ]:
# Connect UHFQA to the PQSC
uhfqa.enable_qccs_mode()

* Configure the input and output impedances

In [ ]:
# Select the input impedance as 50Ω
uhfqa.sigins[0].imp50(True)
uhfqa.sigins[1].imp50(True)
# Select the load impedance as 50Ω
uhfqa.sigouts[0].imp50(True)
uhfqa.sigouts[1].imp50(True)

* Turn on the output channels
* Run the AWG in single mode

In [ ]:
# Enable outputs
uhfqa.sigouts[0].on(True)
uhfqa.sigouts[1].on(True)
# Disable rerun
uhfqa.awgs[0].single(True)

* Helper function to generate flattop gaussian envelopes for readout pulses and integrations weights

In [ ]:
# Envelope function, with gaussian rise/fall and flattop
def envelope_function(length, rise, a=1.0, sigmas=3):
    x_risefall = np.arange(-rise, rise, 1)
    sigma_wave = rise / sigmas
    risefall_w = np.exp(-((x_risefall) ** 2) / (2 * sigma_wave ** 2))
    length_corr = length - 2 * rise
    flattop = np.ones(length_corr)
    res = a * np.concatenate((risefall_w[:rise], flattop, risefall_w[rise:]), axis=None)
    # plt.plot(res)
    return res

* Quantum Analyzer settings:
    * Bypass deskew and crosstalk matrix since they are not necessary for simulated readout
    * Do not bypass the rotation, it is necessary
    * Set QA delay adjustment to 0
    * Set result source to thresholding

In [ ]:
# Bypass all the non-necessary units
uhfqa.qas[0].bypass.deskew(True)
uhfqa.qas[0].bypass.crosstalk(True)
# Don't bypass rotation
uhfqa.qas[0].bypass.rotation(False)
# Set correct QA delay
uhfqa.qas[0].adjusted_delay(0)
# Set QA Result source
uhfqa.qas[0].result.source("result_after_threshold_unit")

* Readout channel settings:
    * Enable weighted integration mode on all readout channels
    * Set Integration length (equal to the readout pulse length)
    * Set threshold level for all channels
    * Set the integrations weights with the envelope generated with the helper function `envelope_function`
    * Set rotation coefficients of all channels. Since UHFQA works with real numbers, the rotation should bring all results to the real axis of the IQ plane

In [ ]:
# Enable weighted integration mode
uhfqa.qas[0].integration.mode('normal')
# Set Integration length
uhfqa.qas[0].integration.length(readout_pulse_length)
# Set threshold level for all channels
for i in range(num_readout_channels):
    uhfqa.qas[0].thresholds[i].level(0.0)

# Set the integration weights
envelope = envelope_function(readout_pulse_length, rise_fall_length)
x = np.arange(0, readout_pulse_length, 1)
for i, freq in enumerate(frequencies):
    weights_real = envelope * np.cos(2 * np.pi * freq * x / SR_UHFQA)
    weights_imag = envelope * np.sin(2 * np.pi * freq * x / SR_UHFQA)

    uhfqa.qas[0].integration.weights[i].real(weights_real)
    uhfqa.qas[0].integration.weights[i].imag(weights_imag)

# Set rotation coefficients of all channels
for i, freq in enumerate(frequencies):
    uhfqa.qas[0].rotations[i].value(1 - 1j)

* Configure arguments to use with `startQA` function
    * `trigger_int_unit`: Determines which integration units should be triggered
    * `trigger_qa_input`: Determines if QA Monitor should be triggered
    * `readout_register_address`: Determines the address of the register inside the PQSC to write the results in


In [ ]:
# Parameters of UHFQA sequencer
trigger_int_unit = "|".join(f"QA_INT_{i:d}" for i in range(num_readout_channels))
trigger_qa_input = "true"
readout_register_address = register_num

* Sequence program which waits for starting trigger from the PQSC, plays the readout pulse, starts quantum analyzer and waits for a while until feedback processing is complete

In [ ]:
# Sequence for the UHFQA
uhfqa_consts = f"""\
//Constants definition
const WFM_LEN = {readout_pulse_length:d};
const QA_INTS = {trigger_int_unit:s};
const QA_MONITOR = {trigger_qa_input:s};
const READOUT_REG_ADDRESS = {readout_register_address:d};

""" 

uhfqa_program = uhfqa_consts + """\
//Define waveforms for readout
"""

# Add a waveform placeholder for each simulated response
for i in range(num_readouts):
    uhfqa_program += textwrap.dedent(
        f"""\
    wave w{i:d}_I = placeholder(WFM_LEN);
    wave w{i:d}_Q = placeholder(WFM_LEN);
    assignWaveIndex(w{i:d}_I, w{i:d}_Q, {i:d});
    """
    )

uhfqa_program += """\

//Start the main program
"""

for i in range(num_readouts):
    uhfqa_program += textwrap.dedent(
        f"""\
    waitZSyncTrigger();  //Wait for start trigger from PQSC
    playWave(1, w{i:d}_I, 2, w{i:d}_Q);  //Play a simulated response
    startQA(QA_INTS, QA_MONITOR, READOUT_REG_ADDRESS);  //Trigger a readout
    waitZSyncTrigger();  //Wait for the feedback trigger (and ignore it)
    
    """
    )

* Compile the sequence program and upload it to the instrument

In [ ]:
uhfqa.awgs[0].load_sequencer_program(uhfqa_program)

* Generating the readout pulses and uploading them to the instrument:
    * The for loop cycles through all combinations of the phases [0, 180] to generate simulated responses

In [ ]:
# Generate waves
x = np.arange(0, readout_pulse_length, 1)
envelope = envelope_function(readout_pulse_length, rise_fall_length)
waves = Waveforms()

# Cycle through phases [0, 180] to generate simulated responses
for i, phs in enumerate(itertools.product(phases, repeat=num_readout_channels)):
    wave_I = np.zeros(readout_pulse_length)
    wave_Q = np.zeros(readout_pulse_length)

    for freq, ph in zip(frequencies, phs):
        wave_I += np.cos(2 * np.pi * freq / SR_UHFQA * x + np.deg2rad(ph))
        wave_Q += np.sin(2 * np.pi * freq / SR_UHFQA * x + np.deg2rad(ph))

    wave_I *= envelope / num_readouts
    wave_Q *= envelope / num_readouts

    waves[i] = (wave_I, wave_Q)

# Upload waveforms
uhfqa.awgs[0].write_to_waveform_memory(waves)

### 8. Main Program <a class="anchor" id="main"></a>

Final check before running the main program:
* Are the connections on ZSync ports fine?

In [ ]:
# Check ZSync connections
pqsc.check_zsync_connection(ports=zsync_ports,timeout=30)

Stop all AWGs in case a previous measurement is still going on

In [ ]:
# Stop all AWGs
awg_D1.enable(False)
awg_D2.enable(False)
awg_AZ.enable(False)
awg_AX.enable(False)
uhfqa.awgs[0].enable(False)

Arm UHFQA for readout, it will wait to be triggered by the AWG sequencer of the UHFQA

In [ ]:
# Arm UHFQA readout
uhfqa.qas[0].result.length(num_readouts)
uhfqa.qas[0].result.averages(1)
uhfqa.qas[0].result.reset(True)

Start all AWGs, they will wait for triggers from PQSC

In [ ]:
# Run all AWGs
awg_D1.enable(True)
awg_D2.enable(True)
awg_AZ.enable(True)
awg_AX.enable(True)
uhfqa.awgs[0].enable(True)
session.sync()

Finally, start sending out triggers from the PQSC

In [ ]:
# Start PQSC
pqsc.arm_and_run()